# A multicommodity transportation model

In [1]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Note: you may need to restart the kernel to use updated packages.


The transportation model of the previous chapter was concerned
with shipping a single commodity from origins to destinations.  Suppose
now that we are shipping several different products.  We can define a new
`set`,
`PROD` ,
whose members represent the different products, and we can
add
`PROD`
to the indexing of every component in the model;
the result can be seen in Figure @Tut4@-@multi.mod@.

In [2]:
%%writefile multi.mod

set ORIG;   # origins
set DEST;   # destinations
set PROD;   # products

param supply {ORIG,PROD} >= 0;  # amounts available at origins
param demand {DEST,PROD} >= 0;  # amounts required at destinations

   check {p in PROD}:
      sum {i in ORIG} supply[i,p] = sum {j in DEST} demand[j,p];

param limit {ORIG,DEST} >= 0;

param cost {ORIG,DEST,PROD} >= 0;  # shipment costs per unit
var Trans {ORIG,DEST,PROD} >= 0;   # units to be shipped

minimize Total_Cost:
   sum {i in ORIG, j in DEST, p in PROD}
      cost[i,j,p] * Trans[i,j,p];

subject to Supply {i in ORIG, p in PROD}:
   sum {j in DEST} Trans[i,j,p] = supply[i,p];

subject to Demand {j in DEST, p in PROD}:
   sum {i in ORIG} Trans[i,j,p] = demand[j,p];

subject to Multi {i in ORIG, j in DEST}:
   sum {p in PROD} Trans[i,j,p] <= limit[i,j];

Overwriting multi.mod


In [13]:
ORIG = ['GARY', 'CLEV', 'PITT']
DEST = ['FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
PROD = ['bands', 'coils', 'plate']

limit = [625 for i in ORIG for j in DEST]

df_supply = pd.DataFrame(
    [
        ['GARY', 400, 800, 200],
        ['CLEV', 700, 1600, 300],
        ['PITT', 800, 1800, 300]
    ],
    columns = ['ORIG', 'bands', 'coils', 'plate']
).set_index('ORIG')

df_demand = pd.DataFrame(
    [
        ['FRA', 300, 500, 100],
        ['DET', 300, 750, 100],
        ['LAN', 100, 400, 0],
        ['WIN', 75, 250, 50],
        ['STL', 650, 950, 200],
        ['FRE', 225, 850, 100],
        ['LAF', 250, 500, 250]
    ],
    columns = ['DEST', 'bands', 'coils', 'plate']
).set_index('DEST')

df_cost = pd.DataFrame(
    [
        ['GARY', 'FRA', 30, 39, 41],
        ['GARY', 'DET', 10, 14, 15],
        ['GARY', 'LAN', 8, 11, 12],
        ['GARY', 'WIN', 10, 14, 16],
        ['GARY', 'STL', 11, 16, 17],
        ['GARY', 'FRE', 71, 82, 86],
        ['GARY', 'LAF', 6, 8, 8],
        ['CLEV', 'FRA', 22, 27, 29],
        ['CLEV', 'DET', 7, 9, 9],
        ['CLEV', 'LAN', 10, 12, 13],
        ['CLEV', 'WIN', 7, 9, 9],
        ['CLEV', 'STL', 21, 26, 28],
        ['CLEV', 'FRE', 82, 95, 99],
        ['CLEV', 'LAF', 13, 17, 18],
        ['PITT', 'FRA', 19, 24, 26],
        ['PITT', 'DET', 11, 14, 14],
        ['PITT', 'LAN', 12, 17, 17],
        ['PITT', 'WIN', 10, 13, 13],
        ['PITT', 'STL', 25, 28, 31],
        ['PITT', 'FRE', 83, 99, 104],
        ['PITT', 'LAF', 15, 20, 20]
    ],
    columns = ['ORIG', 'DEST', 'bands', 'coils', 'plate']
).set_index(['ORIG', 'DEST'])

display(df_supply)
display(df_demand)
display(df_cost)

,bands,coils,plate
ORIG,,,
GARY,400,800,200
CLEV,700,1600,300
PITT,800,1800,300


,bands,coils,plate
DEST,,,
FRA,300,500,100
DET,300,750,100
LAN,100,400,0
WIN,75,250,50
STL,650,950,200
FRE,225,850,100
LAF,250,500,250


bands  coils  plate
ORIG DEST                     
GARY FRA      30     39     41
     DET      10     14     15
     LAN       8     11     12
     WIN      10     14     16
     STL      11     16     17
     FRE      71     82     86
     LAF       6      8      8
CLEV FRA      22     27     29
     DET       7      9      9
     LAN      10     12     13
     WIN       7      9      9
     STL      21     26     28
     FRE      82     95     99
     LAF      13     17     18
PITT FRA      19     24     26
     DET      11     14     14
     LAN      12     17     17
     WIN      10     13     13
     STL      25     28     31
     FRE      83     99    104
     LAF      15     20     20

Because
`supply` ,
`demand` ,
`cost` ,
and
`Trans`
are indexed over one more set in
this version, they take one more subscript:
`supply[i,p]`
for the
amount of product
`p`
shipped from origin
`i` ,
`Trans[i,j,p]`
for the amount of
`p`
shipped from
`i`
to
`j` ,
and so forth.  Even the
`check`
statement is now indexed
over
`PROD` ,
so that it verifies that supply equals demand for each separate
product.

If we look at 
`Supply` ,
`Demand`
and
`Trans` ,
there are (origins + destinations) $\times$ (products) constraints in
(origins) $\times$ (destinations) $\times$ (products) variables.  The result could be quite a
large linear program, even if the individual sets do not have many
members.  For example, 5 origins, 20 destinations and 10 products give
250 constraints in 1000 variables.  The size of this LP is misleading,
however, because the shipments of the products are independent.  That is,
the amounts we ship of one product do not affect the amounts we can ship
of any other product, or the costs of shipping any other product.  We would
do better in this case to solve a smaller transportation problem for each
individual product.  In `AMPL` terms, we would use the simple
transportation model from the previous chapter, together with a different
data file for each product.

The situation would be different if some additional circumstances
had the effect of tying together the different products.  As an example,
imagine that there are restrictions on the
total shipments of products from an origin to a destination,
perhaps because of limited shipping capacity.  To
accommodate such restrictions in our model, we declare a new
parameter
`limit`
indexed over the combinations of origins and
destinations:

```
param limit {ORIG,DEST} >= 0;
```

Then we have a new collection of (origins) $\times$ (destinations) constraints, one for each origin
`i`
and
destination
`j` ,
which say that the sum of shipments from
`i`
to
`j`
of all
products
`p`
may not exceed
`limit[i,j]` :

```
subject to Multi {i in ORIG, j in DEST}:
   sum {p in PROD} Trans[i,j,p] <= limit[i,j];
```

Subject to these constraints (also shown in Figure @Tut4@-@multi.mod@),
we can no longer set the amount of one
product shipped from
`i`
to
`j`
without considering the amounts of other
products also shipped from
`i`
to
`j` ,
since it is the sum of all products that is limited.  Thus
we have no choice but to solve the one large linear program.

For the steel mill in [Tutorial 1](../01/01.md), the products were bands,
coils, and plate.  Thus the data for the multicommodity model could look
like Figure @Tut4@-@multi.dat@.
We invoke `AMPL` in the usual way to get the following solution:

In [37]:
ampl = AMPL()
ampl.read('multi.mod')
ampl.set['ORIG'] = ORIG
ampl.set['DEST'] = DEST
ampl.set['PROD'] = PROD

ampl.param['supply'] = df_supply
ampl.param['demand'] = df_demand
ampl.param['cost'] = df_cost
ampl.param['limit'] = limit

ampl.option['solver'] = 'highs'
ampl.solve()

df_trans = ampl.var['Trans'].to_pandas()

df_trans.columns = ["Trans"]
df_trans.index.names = ['ORIG', 'DEST', 'PROD']
#df_trans = df_trans.unstack('DEST')

display(df_trans)

for p in PROD:
    #q = 'PROD == ' + p
    #df_temp = df_trans.loc[('PROD', p)]
    #df_temp = df_trans[df_trans['PROD']==p]
    df_temp = df_trans.loc[:, :, p]
    df_temp.columns = [p]
    #df_temp.unstack()
    #print("Transportation table for", p)
    display(df_temp.unstack())



HiGHS 1.6.0: HiGHS 1.6.0: optimal solution; objective 199500
45 simplex iterations
0 barrier iterations
 


Trans
ORIG DEST PROD        
CLEV DET  bands      0
          coils    525
          plate    100
     FRA  bands    275
          coils      0
...                ...
PITT STL  coils    625
          plate      0
     WIN  bands      0
          coils      0
          plate      0

[63 rows x 1 columns]

bands                             
DEST   DET  FRA  FRE  LAF  LAN  STL WIN
ORIG                                   
CLEV     0  275    0  100    0  250  75
GARY     0    0    0    0    0  400   0
PITT   300   25  225  150  100    0   0

coils                              
DEST   DET  FRA  FRE  LAF  LAN  STL  WIN
ORIG                                    
CLEV   525    0   50   75  400  300  250
GARY     0    0  625  150    0   25    0
PITT   225  500  175  275    0  625    0

plate                            
DEST   DET  FRA  FRE  LAF LAN  STL WIN
ORIG                                  
CLEV   100    0  100   50   0    0  50
GARY     0    0    0    0   0  200   0
PITT     0  100    0  200   0    0   0

In [5]:
.F1
.P1
.get multi.dat
.P2
.C "Figure @Tut4@-@multi.dat@" "Multicommodity transportation problem data (\f(CWmulti.dat\fP).
.ix file~[multi.dat]
.F2


.P1
ampl: \f(CXmodel multi.mod; data multi.dat; solve;\fP
CPLEX 8.0.0: optimal solution; objective 199500
41 dual simplex iterations (0 in phase I)
.P2
.P1
ampl: \f(CXdisplay {p in PROD}: {i in ORIG, j in DEST} Trans[i,j,p];\fP

Trans[i,j,'bands'] [*,*] (tr)
:    CLEV  GARY  PITT    :=
DET     0     0   300
FRA   225     0    75
FRE     0     0   225
LAF   225     0    25
LAN     0     0   100
STL   250   400     0
WIN     0     0    75
;
.P2
.P1
Trans[i,j,'coils'] [*,*] (tr)
:    CLEV  GARY  PITT    :=
DET   525     0   225
FRA     0     0   500
FRE   225   625     0
LAF     0   150   350
LAN   400     0     0
STL   300    25   625
WIN   150     0   100
;
.P2
.P1
Trans[i,j,'plate'] [*,*] (tr)
:    CLEV  GARY  PITT    :=
DET   100     0     0
FRA    50     0    50
FRE   100     0     0
LAF     0     0   250
LAN     0     0     0
STL     0   200     0
WIN    50     0     0
;
.P2


SyntaxError: unterminated string literal (detected at line 5) (1171255758.py, line 5)

In both our specification of the shipping costs and `AMPL`'s display of the
solution, a three-dimensional collection of data (that is,
indexed over three sets) must be represented on a two-dimensional screen
or page.  We accomplish this by "slicing" the data along one index, so that
it appears as a collection of two-dimensional tables.  The
`display`
command will make a guess as to the best index on which to slice,
but by
use of an explicit indexing expression as shown above, we can tell it to display
a table for each product.

The optimal solution above ships only 25 tons of coils from *GARY* to
*STL* and 25 tons of bands from *PITT* to *LAF*.
It might be reasonable to require that, if any amount at all is shipped,
it must be at least, say, 50 tons.  In terms
of our model, either
`Trans[i,j,p] = 0`
or
`Trans[i,j,p] >= 50` .
Unfortunately, although it is possible to write such an "either/or"
constraint in `AMPL`,
it is not a linear constraint, and so
there is no way that an LP solver can handle it.
Chapter @Integer@ explains how more powerful (but costlier) integer programming
techniques can deal with this and related kinds of discrete restrictions.